# MVD 7. cvičení

## 1. část - Vytvoření dat

V dnešním cvičení je před implementací shlukování potřeba vygenerovat vlastní data. Cílem je vytvořit několik datasetů dle předlohy a vizualizovat je. Postačí dosažení podobného pozice a rozložení bodů.

### 1. dataset - 2 třídy, 100 datových bodů
Použijte funkci ```make_blobs``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html).

![dataset1](expected_outputs/dataset1.png)

Připravený dataset pro další úlohu (k-means) může vypadat například takto:
![prepared](expected_outputs/prepared_dataset.png)

In [187]:
from sklearn.datasets import make_blobs, make_moons, make_circles
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

In [68]:
x1, y1 = make_blobs(n_samples=100, centers=2, n_features=2)
df1 = pd.DataFrame({'x': x1[:,0], 'y': x1[:,1],"label":y1,"kmeans":["TODO"]*100})
fig = px.scatter(df1,x="x",y="y", color=['osoba ' + str(x)  for x in y1])
fig.show()

### 2. dataset - 4 třídy, 200 datových bodů
Použijte funkci ```make_blobs``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html).

![dataset1](expected_outputs/dataset2.png)

In [69]:
x2, y2 = make_blobs(n_samples=200, centers=4, n_features=2)
df2 = pd.DataFrame({'x': x2[:,0], 'y': x2[:,1],"label":y2,"kmeans":["TODO"]*200})
fig = px.scatter(df2,x="x",y="y", color=[str(x)  for x in y2])
fig.show()

### 3. dataset - 2 třídy, 200 datových bodů
Použijte funkci ```make_moons``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html#sklearn.datasets.make_moons).

![dataset1](expected_outputs/dataset3.png)

In [70]:
x3, y3 = make_moons(n_samples=(100,100), noise=0.075)
df3 = pd.DataFrame({'x': x3[:,0], 'y': x3[:,1],"label":y3,"kmeans":["TODO"]*200})
fig = px.scatter(df3,x="x",y="y", color=[str(x)  for x in y3])
fig.show()

### 4. dataset - 2 třídy, 200 datových bodů
Použijte funkci ```make_circles``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html#sklearn.datasets.make_circles).

![dataset1](expected_outputs/dataset4.png)

In [71]:
x4, y4 = make_circles(n_samples=(100,100), noise=0.075,factor=0.2)
df4 = pd.DataFrame({'x': x4[:,0], 'y': x4[:,1],"label":y4,"kmeans":["TODO"]*200})
fig = px.scatter(df4,x="x",y="y", color=[str(x)  for x in y4])
fig.show()

## 2. část - K-means

Implementujte K-means algoritmus, aplikujte ho na všechny 4 datasety a zobrazte výsledky algoritmu. Zároveň se pokuste zvýraznit centroid ve vizualizaci výsledku. Pro aplikaci K-means můžete použít počet tříd v každém datasetu (není třeba hledat parametr K automaticky).

In [88]:
import numpy as np

In [235]:
def kmean(db:pd.DataFrame,it = 100, eps = 0.001) -> tuple:


    k = len(db['label'].unique()) # number of labels
    p = db[["x","y"]].sample(n=k)
    centroids = np.array([np.array(p["x"]),np.array(p["y"])]).T #init centroids
    data = np.array([np.array(db["x"]),np.array(db["y"])]).T #init data Matrix
    distance = np.zeros((db["x"].shape[0],k)) # init distance
    for i in range(it):
        for j in range(k): # K iteration
            distance[:,j] = np.sum(((centroids[j]-data)**2), axis=1)

        db["kmeans"] = np.argmin(distance, axis=1)
        centroids =  db.groupby(['kmeans']).mean()
        centroids = np.array([np.array(centroids["x"]),np.array(centroids["y"])]).T


    return db, centroids.T



In [237]:
for df_x in [df1,df2,df3,df4]:
    df_kmean,centroids = kmean(df_x)
    fig = px.scatter(df_kmean,x="x",y="y", color=df_kmean["kmeans"].astype(str))
    fig.add_trace(
    go.Scatter(
        x=centroids[0],
        y=centroids[1],
        mode="markers",
        marker=dict(
            color='black',
            size=10),
        showlegend=False
    )

    )

    fig.show()

## Bonus - K-means s nalezenou hodnotou K

Zkuste najít algoritmy, které umožňují hledat hodnotu K pro K-means. Implementujte alespoň jeden z nich a ověřte, že funguje. Ověření proveďte nejen na našich předchozích datasetech, ale zkuste si vygenerovat i dataset s větším množstvím tříd (10-20). (1 bod)

Za implementaci více algoritmů, případně za hlubší analýzu výsledků (např. kdy už přestává hledání hodnoty K správně fungovat) je možné získat ještě jeden bonusový bod.

## Elbow method

- spočítat kvadrát odchylek jednotlivých bodů od svého centroidu
- provést pro 1 až maxK kmeans
- vykreslit do grafu a nalézt tzv. "loket"


## Silhouete method